# step2_agents_in_a_workflow.py - ELI5 Walkthrough
This notebook mirrors `python/samples/getting_started/workflows/_start-here/step2_agents_in_a_workflow.py` with commentary between each block.


## Big Picture
A Writer agent drafts a slogan, then a Reviewer agent critiques it. The workflow simply connects the two and prints what each agent says along with the final output list.


## Key Ingredients
- Azure-hosted chat agents created via `AzureOpenAIChatClient`.
- `WorkflowBuilder` wiring without custom executors—agents themselves can act as nodes.
- Event iteration over `AgentRunEvent` objects to display intermediate agent messages.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["User Prompt"]) --> Writer[[Writer Agent]]
    Writer --> Reviewer[[Reviewer Agent]]
    Reviewer --> Output(["Workflow Outputs"])
```


### Step 1: Imports and scenario overview
Load dotenv, asyncio, and the Agent Framework pieces, then keep the docstring that explains why two agents are wired together.


In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from agent_framework import AgentRunEvent, WorkflowBuilder
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
load_dotenv()

"""
Step 2: Agents in a Workflow non-streaming

This sample uses two custom executors. A Writer agent creates or edits content,
then hands the conversation to a Reviewer agent which evaluates and finalizes the result.

Purpose:
Show how to wrap chat agents created by AzureOpenAIChatClient inside workflow executors. Demonstrate how agents
automatically yield outputs when they complete, removing the need for explicit completion events.
The workflow completes when it becomes idle.

Prerequisites:
- Azure OpenAI configured for AzureOpenAIChatClient with required environment variables.
- Authentication via azure-identity. Use AzureCliCredential and run az login before executing the sample.
- Basic familiarity with WorkflowBuilder, executors, edges, events, and streaming or non streaming runs.
"""




### Step 2: Build and run the agent workflow
`main()` spins up writer/reviewer agents, builds the graph, runs it once, and prints both agent events and the collected outputs.


In [ ]:
async def main():
    """Build and run a simple two node agent workflow: Writer then Reviewer."""
    # Create the Azure chat client. AzureCliCredential uses your current az login.
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    writer_agent = chat_client.create_agent(
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
        name="writer",
    )

    reviewer_agent = chat_client.create_agent(
        instructions=(
            "You are an excellent content reviewer."
            "Provide actionable feedback to the writer about the provided content."
            "Provide the feedback in the most concise manner possible."
        ),
        name="reviewer",
    )

    # Build the workflow using the fluent builder.
    # Set the start node and connect an edge from writer to reviewer.
    workflow = WorkflowBuilder().set_start_executor(writer_agent).add_edge(writer_agent, reviewer_agent).build()

    # Run the workflow with the user's initial message.
    # For foundational clarity, use run (non streaming) and print the terminal event.
    events = await workflow.run("Create a slogan for a new electric SUV that is affordable and fun to drive.")
    # Print agent run events and final outputs
    for event in events:
        if isinstance(event, AgentRunEvent):
            print(f"{event.executor_id}: {event.data}")

    print(f"{'=' * 60}\nWorkflow Outputs: {events.get_outputs()}")
    # Summarize the final run state (e.g., COMPLETED)
    print("Final state:", events.get_final_state())

    """
    Sample Output:

    writer: "Charge Up Your Adventure—Affordable Fun, Electrified!"
    reviewer: Slogan: "Plug Into Fun—Affordable Adventure, Electrified."

    **Feedback:**
    - Clear focus on affordability and enjoyment.
    - "Plug into fun" connects emotionally and highlights electric nature.
    - Consider specifying "SUV" for clarity in some uses.
    - Strong, upbeat tone suitable for marketing.
    ============================================================
    Workflow Outputs: ['Slogan: "Plug Into Fun—Affordable Adventure, Electrified."

    **Feedback:**
    - Clear focus on affordability and enjoyment.
    - "Plug into fun" connects emotionally and highlights electric nature.
    - Consider specifying "SUV" for clarity in some uses.
    - Strong, upbeat tone suitable for marketing.']
    """




### Step 3: Try it yourself
Use the helper below. In notebooks it awaits `main()` on the active loop; in scripts it falls back to `asyncio.run(main())`.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())
